# Goal

I try to train a SVM on the MNIST dataset reducing the dimensions with PCA to a number that I can manage in some minutes with my notebook.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

from time import time

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

import math

# Some helper functions first 

I prepared a couple of functions to see the pictures before and after the dimensionality reduction.

In [ ]:
# Plots the image represented by a row
def plot_number(row, w=28, h=28, labels=True):
    if labels:
        # the first column contains the label
        label = row[0]
        # The rest of columns are pixels
        pixels = row[1:]
    else:
        label = ''
        # The rest of columns are pixels
        pixels = row[0:]
    
#    print(row.shape, pixels.shape)
        

    # Make those columns into a array of 8-bits pixels
    # This array will be of 1D with length 784
    # The pixel intensity values are integers from 0 to 255
    pixels = 255-np.array(pixels, dtype='uint8')

    # Reshape the array into 28 x 28 array (2-dimensional array)
    pixels = pixels.reshape((w, h))

    # Plot
    if labels:
        plt.title('Label is {label}'.format(label=label))
    plt.imshow(pixels, cmap='gray')

# Plots a whole slice of pictures
def plot_slice(rows, size_w=28, size_h=28, labels=True):
    num = rows.shape[0]
    w = 4
    h = math.ceil(num / w)
    fig, plots = plt.subplots(h, w)
    fig.tight_layout()

    for n in range(0, num):
        s = plt.subplot(h, w, n+1)
        s.set_xticks(())
        s.set_yticks(())
        plot_number(rows.ix[n], size_w, size_h, labels)
    plt.show()


### Loading the training set
Let's load the dataset and see some basic info.

In [ ]:
train = pd.read_csv('../input/train.csv')
#train = pd.read_csv('train.csv')
print(train.shape)
train.sample(5)


In [ ]:
train.describe()

In [ ]:
train.info()

### Visualizing some rows

Plotting a slice to see correspondence between labels and pictures.

In [ ]:
# Plotting first few rows
plot_slice(train[0:12])


## Dimensionality reduction

I try to see if a PCA can help to reduce the number of features without affecting too much information. First of all I split the whole dataset in features and class, then I prepare the split of both sets to training and test set.

In [ ]:
X_train = train.drop(['label'], axis='columns', inplace=False)
y_train = train['label']

from sklearn.model_selection import train_test_split
X_tr, X_ts, y_tr, y_ts = train_test_split(X_train, y_train, test_size=0.30, random_state=4)


The `n_components` variable here is crucial. With a value of 16 the model can be trained in seconds and still achieve a `0.97229` score in the Leaderboard. I use a perfect square value because I want to see how it looks the reduced-dimension image.

I've seen that using 49 components resulted in a smaller score, maybe that means that the final model is more over-fitted than the 16 components one.

In [ ]:
n_components = 16
t0 = time()
pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=True).fit(X_train)
print("done in %0.3fs" % (time() - t0))

X_train_pca = pca.transform(X_train)


Let's see the variance histogram.

In [ ]:
plt.hist(pca.explained_variance_ratio_, bins=n_components, log=True)
pca.explained_variance_ratio_.sum()


Even though 16 components collect 59% of variance and 49 components get 82%, the former performs better at submission.

Let's see how the smaller pictures look.

In [ ]:
plot_slice(pd.DataFrame(data=X_train_pca[0:12]), size_w=4, size_h=4, labels=False)

Hope someone could explain these spots :D

I've removed the real grid search parameters here because it takes too long to make it online

In [ ]:
param_grid = { "C" : [0.1]
              , "gamma" : [0.1]}
rf = SVC()
gs = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='accuracy', cv=2, n_jobs=-1, verbose=1)
gs = gs.fit(X_train_pca, y_train)

print(gs.best_score_)
print(gs.best_params_)


In [ ]:
bp = gs.best_params_

t0 = time()
clf = SVC(C=bp['C'], kernel='rbf', gamma=bp['gamma'])
clf = clf.fit(X_train_pca, y_train)
print("done in %0.3fs" % (time() - t0))


In [ ]:
clf.score(pca.transform(X_ts), y_ts)

Nice score with a little effort.

In [ ]:
#val = pd.read_csv('test.csv')
val = pd.read_csv('../input/test.csv')
pred = clf.predict(pca.transform(val))
# ImageId,Label

val['Label'] = pd.Series(pred)
val['ImageId'] = val.index +1
sub = val[['ImageId','Label']]

sub.to_csv('submission.csv', index=False)

## Conclusions

It's been a very interesting experiment on how to reduce the features number to train the model in a decent time and still keep a goode score. It's good to see that I could reduce the number of features from __784 to 16__ with a __97%__ of overall score.

A next step could be using a Neural Network to see if there are improvements in the training time or score.

Any comment is appreciated!